In [1]:
import re
import numpy as np

In [2]:
# RefSeq: NC_000001.11
file = open('sequence2.fasta')
seq = file.read()
seq = re.sub(r"\n", "", seq)
seq = seq[68:]

In [3]:
def f(seq, A, B):
    count_A = 0
    count_B = 0
    for i in seq:
        if(A == i):
            count_A += 1
        if(B == i):
            count_B += 1
    return (count_A + count_B)/len(seq)

def C(seq, A, B, ff):
    count_AB = 0
    for i in range(len(seq)-1):
        if(seq[i]+seq[i+1] == A+B):
            count_AB += 1
    expected = Expected(seq, A, B, ff)
    if(abs(expected) < 1e-16 and count_AB == 0):
        return 0.5
    elif(abs(expected) < 1e-16):
        return 100
    return count_AB/expected

def Expected(seq, A, B, ff):
    tmp = f(seq, A, B)
    ff.append(tmp)
    return (tmp**2)*len(seq)

def IsUndeOverRepresentationIsland(seq, left_border, right_border, A, B, ff):
    return C(seq[left_border:right_border], A, B, ff)
    
    
def UndeOverRepresentationIslands(seq, length, A, B):
    left_border = 0
    right_border = length
    under_representation = []
    over_representation = []
    ff = []
    
    for i in range(len(seq)//length):
        C = IsUndeOverRepresentationIsland(seq, left_border, right_border, A, B, ff)
        if(C > 0.7 and abs(C-100) > 1e-16):
            over_representation.append([left_border, right_border, C])
        elif(C < 0.37):
            under_representation.append([left_border, right_border, C])
    
        left_border = right_border
        if(right_border+length < len(seq)):
            right_border = right_border+length
        else:
            right_border = len(seq)
        #print('[' + str(left_border) + ',' + str(right_border) + ']')
    return under_representation, over_representation, ff

In [4]:
under, over, ff = UndeOverRepresentationIslands(seq, 1000, 'T','A')

In [5]:
np.array(under)

array([[1.10000000e+04, 1.20000000e+04, 1.86737787e-01],
       [1.20000000e+04, 1.30000000e+04, 1.25000000e-01],
       [1.30000000e+04, 1.40000000e+04, 1.16262976e-01],
       ...,
       [2.48942000e+08, 2.48943000e+08, 1.82067887e-01],
       [2.48943000e+08, 2.48944000e+08, 1.82392303e-01],
       [2.48944000e+08, 2.48945000e+08, 2.24965744e-01]])

In [6]:
np.array(over)

array([[3.47000000e+05, 3.48000000e+05, 2.50000000e+00],
       [1.67990000e+07, 1.68000000e+07, 1.21687399e+00],
       [2.95530000e+07, 2.95540000e+07, 1.11383382e+00],
       [1.25130000e+08, 1.25131000e+08, 8.81269027e-01],
       [1.25131000e+08, 1.25132000e+08, 1.24947938e+00],
       [2.23608000e+08, 2.23609000e+08, 3.82653061e+00],
       [2.48946000e+08, 2.48947000e+08, 1.50000000e+00]])

Найдем среднюю предполагаемую частоту TpA

In [8]:
ff = np.array(ff)

In [9]:
ff.mean()

0.5395107609376757

In [10]:
ff.std()

0.1686548536396452

Будет логичным взять значение среднего +- стандартное отклонение в качетсве пороговых значений

In [11]:
ff.mean() + ff.std()

0.7081656145773209

In [12]:
ff.mean() - ff.std()

0.37085590729803053

In [13]:
len(under)

230375

In [14]:
len(over)

7